In [1]:
import numpy as np
import pandas as pd
import os
import os.path as osp

data_dir = r"C:\Users\user\dev\copycat\data\ieee-fraud-detection"
p_trans = osp.join(data_dir, "train_transaction.csv")
p_identity = osp.join(data_dir, "train_identity.csv")

pd.read_csv(p_trans, nrows=5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df_trans = pd.read_csv(p_trans)
df_identity = pd.read_csv(p_identity)

In [3]:
df_merged = pd.merge(df_trans, df_identity, how="outer", on="TransactionID")

In [47]:
# df = df_merged
df = df_trans

In [5]:
X = df.drop(["TransactionID", "isFraud", "TransactionDT"], axis="columns")
y = df["isFraud"]

In [12]:
X["M4"]


0          M2
1          M0
2          M0
3          M0
4         NaN
5          M1
6          M0
7          M0
8         NaN
9          M0
10         M0
11         M0
12         M1
13        NaN
14         M0
15        NaN
16        NaN
17        NaN
18        NaN
19         M1
20         M0
21        NaN
22        NaN
23        NaN
24        NaN
25        NaN
26         M0
27        NaN
28         M1
29         M0
         ... 
590510     M0
590511    NaN
590512     M0
590513    NaN
590514    NaN
590515     M0
590516    NaN
590517     M0
590518     M0
590519    NaN
590520     M0
590521     M2
590522     M0
590523    NaN
590524     M1
590525    NaN
590526    NaN
590527    NaN
590528    NaN
590529     M2
590530    NaN
590531    NaN
590532    NaN
590533    NaN
590534     M2
590535     M0
590536     M0
590537    NaN
590538     M0
590539    NaN
Name: M4, Length: 590540, dtype: object

In [79]:
def preprocess_fraud_data(df):
    cat_fields = ["ProductCD", "card4", "card6", "M4"]
    df_cat = df[cat_fields].copy()
    df_cat = pd.get_dummies(df_cat)

    TrueFalse_fields = ["M1", "M2", "M3", "M5", "M6", "M7", "M8", "M9"]
    df_TF = df[TrueFalse_fields].copy()
    df_TF[~df_TF.isna()] = df_TF[~df_TF.isna()] == 'T'
    df_TF = df_TF.astype(float)

    leakage_fields = ["TransactionID", "isFraud"]
    problematic_fields = ["P_emaildomain", "R_emaildomain", "TransactionDT"]

    X = df.drop(leakage_fields + problematic_fields + TrueFalse_fields + cat_fields, axis="columns")
    X = pd.concat([X, df_cat, df_TF], axis="columns")
    return X

In [80]:
X = preprocess_fraud_data(df_trans)
y = df_trans["isFraud"]

In [82]:
from xgboost_utils import fit_xgboost_classifier
fit_xgboost_classifier(X, y)

KeyboardInterrupt: 

In [15]:
for field in ["ProductCD", "card4", "card6", "P_emaildomain", "R_emaildomain",
              "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9"]:
    print('\n', field, X[field].unique())



 ProductCD ['W' 'H' 'C' 'S' 'R']

 card4 ['discover' 'mastercard' 'visa' 'american express' nan]

 card6 ['credit' 'debit' nan 'debit or credit' 'charge card']

 P_emaildomain [nan 'gmail.com' 'outlook.com' 'yahoo.com' 'mail.com' 'anonymous.com'
 'hotmail.com' 'verizon.net' 'aol.com' 'me.com' 'comcast.net'
 'optonline.net' 'cox.net' 'charter.net' 'rocketmail.com' 'prodigy.net.mx'
 'embarqmail.com' 'icloud.com' 'live.com.mx' 'gmail' 'live.com' 'att.net'
 'juno.com' 'ymail.com' 'sbcglobal.net' 'bellsouth.net' 'msn.com' 'q.com'
 'yahoo.com.mx' 'centurylink.net' 'servicios-ta.com' 'earthlink.net'
 'hotmail.es' 'cfl.rr.com' 'roadrunner.com' 'netzero.net' 'gmx.de'
 'suddenlink.net' 'frontiernet.net' 'windstream.net' 'frontier.com'
 'outlook.es' 'mac.com' 'netzero.com' 'aim.com' 'web.de' 'twc.com'
 'cableone.net' 'yahoo.fr' 'yahoo.de' 'yahoo.es' 'sc.rr.com' 'ptd.net'
 'live.fr' 'yahoo.co.uk' 'hotmail.fr' 'hotmail.de' 'hotmail.co.uk'
 'protonmail.com' 'yahoo.co.jp']

 R_emaildomain [nan 'gmai

In [22]:
y = df["isFraud"]
y.shape

(590540,)

In [10]:
len(df_identity["TransactionID"].unique())

100

In [13]:
len(pd.merge(df_trans, df_identity, how="outer", on="TransactionID"))

180

In [16]:
print(len(df_trans))
print(len(df_identity))
print(len(df_merged))


590540
144233
590540
